### Imports

In [1]:
import os
import cv2
import pickle
from pytesseract import Output, image_to_data
from skimage import io, color
from skimage.filters import threshold_otsu

### Functions

In [2]:
def process_image(image):
    image = color.rgb2gray(image)
    
    thresh = threshold_otsu(image)
    image = image > thresh
    return image

### Get Bounding Boxes

In [3]:
location = './../../temp_data/images/'
output_dir = './../../temp_data/results/tesseract/'
dir_list = os.listdir(location)

In [4]:
dimensions = {}
results = {}
pixels = {}

for image in dir_list:
    values = []
    img = io.imread(location + image)
    d = image_to_data(img, output_type=Output.DICT)
    for i in range(len(d['level'])):
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
        (x, y, w, h) = (int(x), int(y), int(w), int(h))
        values.append([x,y,x+w,y+h])
        if(x==0 and y==0):
            continue
        else:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 0), -1)
    dimensions[image] = values
    results[image] = d
    img = process_image(img)
    img = img*1
    pixels[image] = img
#     io.imsave(output_dir + 'images/' + image, img)

### Store Results

In [5]:
with open(output_dir+'results.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(results, outp, pickle.HIGHEST_PROTOCOL)
    
with open(output_dir+'pixels.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(pixels, outp, pickle.HIGHEST_PROTOCOL)

with open(output_dir+'dimensions.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(dimensions, outp, pickle.HIGHEST_PROTOCOL)